In [1]:
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
import pickle
import random

In [2]:
p=0.4
df = pd.read_csv('env/datasets/cleaned_articles1.csv',header=None,skiprows=lambda i: 1>0 and random.random() > p)

df2 = pd.read_csv('env/datasets/cleaned_articles2.csv',header=None,skiprows=lambda i: 1>0 and random.random() > p)


In [3]:
df.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11
0,2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri...",when walt disney bambi open critic prais spare...
1,3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t...",death may great equal necessarili evenhand of ...
2,4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ...",seoul south korea north korea leader kim said ...
3,12,12,17295,"Mar-a-Lago, the Future Winter White House and ...",New York Times,Maggie Haberman,2017-01-02,2017.0,1.0,NaN,"WEST PALM BEACH, Fla. — When Donald J. Tr...",west palm beach fla when donald j trump rang n...
4,14,14,17297,Turning Your Vacation Photos Into Works of Art...,New York Times,Stephanie Rosenbloom,2017-04-14,2017.0,4.0,NaN,It’s the season for family travel and photos ...,it season famili travel photo perhap enlarg im...


In [4]:
stemmer = SnowballStemmer('english')
words = stopwords.words("english")

In [5]:
df.columns = ['NaN', 'Unnamed', 'id', 'title', 'publication', 'author', 'date', 'year', 'month', 'url', 'content', 'cleaned']
df2.columns = ['NaN', 'Unnamed', 'id', 'title', 'publication', 'author', 'date', 'year', 'month', 'url', 'content', 'cleaned']

In [6]:
df[df.publication == 'Breitbart' & df.Unnamed == '']['cleaned']

TypeError: cannot compare a dtyped [int64] array with a scalar of type [bool]

In [7]:
#Second batch:
n_s_breitbart = df[df.publication == 'Breitbart']
n_s_times = df[df.publication == 'New York Times']
#First batch:
n_s_atlantic = df2[df2.publication == 'Atlantic']
n_s_post = df2[df2.publication == 'New York Post']

In [8]:
n_s = list(n_s_breitbart.iloc[:,11].values) + list(n_s_post.iloc[:,11].values) \
 + list(n_s_atlantic.iloc[:,11].values) + list(n_s_times.iloc[:,11].values)

In [9]:
n_s_breitbart.dropna()

,NaN,Unnamed,id,title,publication,author,date,year,month,url,content,cleaned


In [11]:
n_s_breitbart.iloc[15]["cleaned"]

'marvel comic forc issu apolog follow discoveri alleg refer recent issu artist ardian syaf indonesian artist ardian syaf alleg insert polit religi refer relat recent indonesian elect first issu new comic seri gold syaf work comic report featur refer koran vers al maidah urg muslim take christian jew alli the vers translat o believ take jew christian alli they fact alli one anoth and whoever alli among inde one inde allah guid wrongdo peopl the vers seen shirt charact colossus hundr thousand muslim march christian governor jakarta basuki tjahaja purnama critic polit oppon refer vers as result mani muslim saw act blasphemi call prosecut purnama the comic also referenc number direct refer protest held purnama decemb last year the panel featur number also featur jewish charact kitti pryde stand foreground marvel comic quick respond concern rais notic syaf messag within issu releas statement matter bleed cool the mention artwork gold insert without knowledg behind report mean state marvel c

In [13]:
n_s = [word.replace('New York Post','') for word in n_s]
n_s = [word.replace('Breitbart','') for word in n_s]
n_s = [word.replace('New York Times','') for word in n_s]
n_s = [word.replace('Atlantic','') for word in n_s]

AttributeError: 'float' object has no attribute 'replace'

In [8]:
classes_Bias = np.asarray([1 for i in range(len(n_s_breitbart))] + \
[1 for i in range(len(n_s_post))] + [2 for i in range(len(n_s_atlantic))] + \
[2 for i in range(len(n_s_times))])

In [17]:
X_train, X_test, y_train, y_test = train_test_split(n_s, classes_Bias, test_size=0.2)

pipeline = Pipeline([('vect', TfidfVectorizer(ngram_range=(1, 2), stop_words="english", sublinear_tf=True)),
                     ('chi',  SelectKBest(chi2, k=10000)),
                     ('clf', LinearSVC(C=1.0, penalty='l1', max_iter=3000, dual=False))])

In [18]:
model = pipeline.fit(X_train, y_train)

vectorizer = model.named_steps['vect']
chi = model.named_steps['chi']
clf = model.named_steps['clf']

KeyboardInterrupt: 

In [ ]:
print("accuracy score: " + str(model.score(X_test, y_test)))

In [ ]:
n_s_atlantic.content[7]

In [34]:
print(model.predict(['They were primed to become eukaryotes. ” This debate is likely to rage on for a while, because no one has actually seen the Asgardians themselves. In the 1980s, scientists developed ways of identifying microbes by searching for their DNA amid environmental samples. This is now the main way of finding new species, but it means that many microbes —  like Loki and its kin —  are only known through their DNA. No one knows what they look like. “It’s high on our priority list,” says Ettema. “It’s crucial that we have a look at the cells to see what they’re doing, but that’s extremely hard. Loki was discovered in   ocean sediments, where you don’t go every day. Thor and Heimdall are present in shallower environments, but they’re less than 0. 1 percent of the total microbial community. It’s like looking for a needle in a haystack, but we’re working on it.']))

[2]


In [3]:
df3 = pd.read_pickle('cleaned_articles1.pkl')

In [4]:
df3.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content,cleaned
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...,washington congression republican new fear com...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood...",after bullet shell get count blood dri votiv c...
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri...",when walt disney bambi open critic prais spare...
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t...",death may great equal necessarili evenhand of ...
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ...",seoul south korea north korea leader kim said ...
